<a href="https://colab.research.google.com/github/gustavohroos/treinamento-h2ia/blob/main/Busca-COM-Info/Busca_com_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [173]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [174]:
import numpy as np
import time
from typing import List
from copy import deepcopy
from queue import PriorityQueue
from sys import getsizeof

In [175]:
class Node:
    def __init__(self, puzzle: List[int], parent: object = None) -> None:
        self.children = []
        self.parent = parent
        self.moves = []
        self.puzzle = np.array(puzzle)
        self.h = self.get_heuristic()
        if (self.parent != None):
            self.g = parent.g + 1
        else:
            self.g = 0
        self.f = self.g + self.h

    def __eq__(self, other: object) -> bool:
        return (self.puzzle == other.puzzle).all()

    def __gt__(self, other: object) -> bool:
        return (self.f > other.f)
    
    def __repr__(self) -> str:
        return (str(self.puzzle[0]) + '\n' + 
                str(self.puzzle[1]) + '\n' + 
                str(self.puzzle[2]) + '\n')
    
    def expand_node(self) -> None:
        
        i, j = np.where(self.puzzle == 0)

        if j > 0: #Moves left
            pc = deepcopy(self.puzzle)
            pc[i, j-1], pc[i, j] = pc[i, j], pc[i, j-1]
            self.add_child(pc)

        if j < 2: #Moves right
            pc = deepcopy(self.puzzle)
            pc[i, j+1], pc[i, j] = pc[i, j], pc[i, j+1]
            self.add_child(pc)
        
        if i > 0: #Moves up
            pc = deepcopy(self.puzzle)
            pc[i-1, j], pc[i, j] = pc[i, j], pc[i-1, j]
            self.add_child(pc)
        
        if i < 2: #Moves down
            pc = deepcopy(self.puzzle)
            pc[i+1, j], pc[i, j] = pc[i, j], pc[i+1, j]
            self.add_child(pc)

    def add_child(self, array: np.array) -> None:
        child = Node(array)
        self.children.append(child)
        child.parent = self

    def get_heuristic(self) -> int:
        correct = np.array([[1, 2, 3],
                            [4, 5, 6], 
                            [7, 8, 0]])
        h = 0
        for index1, x1 in np.ndenumerate(correct):
            for index2, x2 in np.ndenumerate(self.puzzle):
                if x1 == x2 and index1 != index2 and x1 != 0:
                    h += abs(index1[0] - index2[0]) + abs(index1[1] - index2[1])
        return h

def path_tracer(node: Node) -> List[Node]:
        current = node
        path = []
        path.append(current)

        while current.parent is not None:
            current = current.parent
            path.append(current)
        
        path.reverse()

        return path

puzzle = [[0,8,7],[6,5,4],[3,2,1]] #Harder
# puzzle = [[1, 2, 3], [4, 0, 6], [7, 5, 8]] #Easier to test

root = Node(puzzle)

## Busca com informação

In [176]:
def astar_search(root: Node) -> List[Node]:
        path = []
        open_list = PriorityQueue()
        open_list.put((root.f, root))
        closed_list = set()
        goal_found = False
        expected = Node([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
        count = 0

        if root == expected:
            goal_found = True

        while open_list and not goal_found:
            f, current_node = open_list.get()
            closed_list.add(str(current_node.puzzle))
            current_node.expand_node()

            for child in current_node.children:
                if child == expected:
                    goal_found = True
                    path = path_tracer(child)
                
                if str(child.puzzle) not in closed_list:
                    open_list.put((child.f, child))
                    count += 1
                    
        return path, count

start = time.time()
solution, count = astar_search(root)

if solution:
    print('Goal found.')
    print(f'It takes {count} nodes.')
    print(f'Solution in {len(solution)} steps.')
    print(solution[0], solution[-1], sep='-------\n')
    print(f'Time taken: {(time.time() - start):.4f} seconds.')
else:
    print('No path to solution is found')

Goal found.
It takes 52 nodes.
Solution in 29 steps.
[0 8 7]
[6 5 4]
[3 2 1]
-------
[1 2 3]
[4 5 6]
[7 8 0]

Time taken: 0.0183 seconds.


In [181]:
print('Steps:')
for i in range(len(solution)):
    print(f'Step {i+1}')
    print(solution[i])

Steps:
Step 1
[0 8 7]
[6 5 4]
[3 2 1]

Step 2
[8 0 7]
[6 5 4]
[3 2 1]

Step 3
[8 7 0]
[6 5 4]
[3 2 1]

Step 4
[8 7 4]
[6 5 0]
[3 2 1]

Step 5
[8 7 4]
[6 5 1]
[3 2 0]

Step 6
[8 7 4]
[6 5 1]
[3 0 2]

Step 7
[8 7 4]
[6 5 1]
[0 3 2]

Step 8
[8 7 4]
[0 5 1]
[6 3 2]

Step 9
[0 7 4]
[8 5 1]
[6 3 2]

Step 10
[7 0 4]
[8 5 1]
[6 3 2]

Step 11
[7 4 0]
[8 5 1]
[6 3 2]

Step 12
[7 4 1]
[8 5 0]
[6 3 2]

Step 13
[7 4 1]
[8 5 2]
[6 3 0]

Step 14
[7 4 1]
[8 5 2]
[6 0 3]

Step 15
[7 4 1]
[8 5 2]
[0 6 3]

Step 16
[7 4 1]
[0 5 2]
[8 6 3]

Step 17
[0 4 1]
[7 5 2]
[8 6 3]

Step 18
[4 0 1]
[7 5 2]
[8 6 3]

Step 19
[4 1 0]
[7 5 2]
[8 6 3]

Step 20
[4 1 2]
[7 5 0]
[8 6 3]

Step 21
[4 1 2]
[7 5 3]
[8 6 0]

Step 22
[4 1 2]
[7 5 3]
[8 0 6]

Step 23
[4 1 2]
[7 5 3]
[0 8 6]

Step 24
[4 1 2]
[0 5 3]
[7 8 6]

Step 25
[0 1 2]
[4 5 3]
[7 8 6]

Step 26
[1 0 2]
[4 5 3]
[7 8 6]

Step 27
[1 2 0]
[4 5 3]
[7 8 6]

Step 28
[1 2 3]
[4 5 0]
[7 8 6]

Step 29
[1 2 3]
[4 5 6]
[7 8 0]



## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

Ao contrário dos algoritmos de busca em largura e busca em profundidade, que, além de demorarem até 48 minutos para encontrar o caminho e utilizarem milhares de nodos, o algoritmo com informação foi capaz de encontrar o mesmo caminho em menos de 1 segundo utilizando apenas 52 nodos. Isso foi possível através da heurística, a qual consegue indicar ao algoritmo qual o próximo passo a ser explorado.

